In [ ]:
# -*- coding: utf-8 -*-
import numpy as np
import pandas as pd

In [ ]:
# Data input
# Data from ensembl-biomart contains ensg, ensp and symbol
# https://www.ensembl.org/biomart/martview
ensp_df = pd.read_csv("")
# Expression matrix, rows are genes, columns are samples
expression_df = pd.read_csv("", index_col=0)

In [ ]:
# Del the version number of ensg
expression_df["Ensembl_ID"] = expression_df["Ensembl_ID"].str.split(".").str[0]

In [ ]:
# Dictionary for ensg to ensp
# Initial screen for ensp_df

ensp_df = ensp_df.dropna()

ensp_df = ensp_df.sort_values(by=["symbol"], ascending=True).reset_index(drop=True)

In [ ]:
# Ensg to symbol (optional)
# Available symbol list
expression_df = expression_df.loc[expression_df["Ensembl_ID"].isin(ensp_df["ensg"])]
expression_df = expression_df.reset_index(drop=True)
symbol = np.array(expression_df["Ensembl_ID"])
for i in expression_df.index:
    symbol[i] = ensp_df.loc[ensp_df["ensg"] == expression_df["Ensembl_ID"][i]][
        "symbol"
    ].values[0]

expression_df = expression_df.sort_index(ascending=True)
expression_df.insert(0, "symbol", symbol)
expression_df.drop(["Ensembl_ID"], axis=1, inplace=True)
# Mean value processing for repeated symbols
expression_df = expression_df.groupby("symbol").mean()
# Drop the genes with more than 10% zero values
result = expression_df[
    (expression_df == 0).sum(axis=1) < expression_df.shape[1] * (1 - 0.9)
]
# Output
result.to_csv("", index=True)

In [ ]:
# Dictionary for ensg, ensp and symbol
# Screen out the needed genes
ensp_df = ensp_df.loc[ensp_df["symbol"].isin(result.index)]
# New column "id" for dictionary
ensp_df["id"] = pd.factorize(ensp_df["symbol"])[0]
# Output
ensp_df.to_csv("", index=False)